In [8]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [9]:
%run ml_project_normalized.ipynb

Note: you may need to restart the kernel to use updated packages.
Unique primary colors:
 1. black
 2. non-black
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
Unique primary colors:
 1. black
 2. non-black
cleaned color
cleaned breed
dropped columns


/var/folders/zg/knpz54sj16988z9vrp2b98fr0000gn/T/ipykernel_13176/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
Unique primary colors:
 1. black
 2. non-black
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [10]:
#kog_reg
nb = nbformat.read("log_reg.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "log_reg_normalized.ipynb")

# Random Frest
nb = nbformat.read("random_forest.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "random_forest_normalized.ipynb")

# X Random Trees
nb = nbformat.read("x_random_trees.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "x_random_trees_normalized.ipynb")

# XGBoost
nb = nbformat.read("xg_boost.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "xg_boost_normalized.ipynb")

# Multiclass OVR XG
nb = nbformat.read("ovr_xg_boost.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "ovr_xg_boost_normalized.ipynb")

In [11]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline  # Important for SMOTE compatibility
from imblearn.over_sampling import SMOTENC

In [ ]:
# rerun when i make changes
# %run log_reg.ipynb
#%run random_forest.ipynb
#%run x_random_trees.ipynb
#%run xg_boost.ipynb
#%run ovr_xg_boost.ipynb
df_train.head()

^C
  File "/opt/anaconda3/lib/python3.12/site-packages/pip/_internal/utils/logging.py", line 13, in <module>
    from pip._vendor.rich.console import (
  File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/rich/console.py", line 62, in <module>
    from .scope import render_scope
  File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/rich/scope.py", line 7, in <module>
    from .table import Table
  File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/rich/table.py", line 17, in <module>
    from ._ratio import ratio_distribute, ratio_reduce
  File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/rich/_ratio.py", line 2, in <module>
    from fractions import Fraction
  File "/opt/anaconda3/lib/python3.12/fractions.py", line 6, in <module>
    from decimal import Decimal
  File "/opt/anaconda3/lib/python3.12/decimal.py", line 102, in <module>
    from _decimal import *
  File "/opt/anaconda3/lib/python3.12/collections/__init__.py", line 441, in namedt

/var/folders/zg/knpz54sj16988z9vrp2b98fr0000gn/T/ipykernel_13176/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
Unique primary colors:
 1. black
 2. non-black
cleaned color
cleaned breed
Done running ml_project.ipynb.
Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']
Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [13]:
def train_voting_classifier(X_train, y_train, X_test, models, voting='soft'):
    """
    Trains a VotingClassifier with the given models and evaluates it on the test data.
    
    Parameters:
    - X_train: Training features
    - y_train: Training labels
    - X_test: Test features
    - models: List of tuples (model_name, model_instance)
    - voting: 'hard' or 'soft' for the type of voting to be used (default is 'soft')
    
    Returns:
    - trained_model: Trained VotingClassifier
    - test_predictions: Predictions on the test set
    """
    # Initialize the VotingClassifier with the given models
    voting_clf = VotingClassifier(estimators=models, voting=voting, n_jobs=-1)
    
    # Fit the model
    voting_clf.fit(X_train, y_train) #   voting_clf.fit_transform(X_test,)
    
    # Make predictions
    test_predictions = voting_clf.predict(X_test)
    
    # Evaluate the model
    accuracy = balanced_accuracy_score(y_train, voting_clf.predict(X_train))
    print(f"Training Accuracy: {accuracy:.4f}")
    
    return voting_clf, test_predictions

# needs CV loop after testing

In [14]:
df_train = bucket_seasons(df_train)
df_test = bucket_seasons(df_test)

df_train = bucket_days(df_train)
df_test = bucket_days(df_test)

df_train['breed'] = df_train['breed'].astype(str)
df_test['breed'] = df_test['breed'].astype(str)

df_train = df_train.drop(columns=['size', 'intake_hour', 'intake_month'])
df_test = df_test.drop(columns=['size', 'intake_hour', 'intake_month'])



# breed is encoded, but dont fret
def encode_breed(val):
    if isinstance(val, int):
        return val  # Keep numeric as-is
    mapping = {'0' : 0, '1' : 1, '2' : 2, '3' : 3, '4' : 4, '5' : 5, '6' : 6, '7' : 7, '8' : 8, 'Common': 9, 'Rare': 10, 'Unknown': 11}
    return mapping.get(val, -1)  # Return -1 for unexpected values (or handle as needed)

# Apply transformation
df_train['breed'] = df_train['breed'].apply(encode_breed)
df_test['breed'] = df_test['breed'].apply(encode_breed)



df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,outcome_type,season,time_of_day
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,non-black,0,Return to Owner,Summer,Afternoon
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,non-black,1,Return to Owner,Spring,Evening
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,9,2022,non-black,0,Transfer,Spring,Night
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,non-black,1,Return to Owner,Winter,Afternoon
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,black,1,Return to Owner,Spring,Morning


In [15]:
df_train['breed'].value_counts()

breed
9     47258
4     13754
0     13398
2      9803
5      9663
3      6505
7      4240
10     2569
8      2163
1      1785
6        15
11        2
Name: count, dtype: int64

In [16]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)




In [17]:
cat_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed','season', 'is_mix', 'time_of_day', 'primary_color'] # experimenting with droping color
num_cols = ['age_upon_intake', 'intake_year']    # Replace with your actual numerical columns
# freq_cols = ['primary_color'] # for trees

'''
# This is just used to get the model with the best classifiers
models = [
    ('logreg', train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0]), #logistic regression
    ('rf', train_RF_classifier(X_train, y_train, X_test, cat_cols)[0]), #random forest
    ('xrt', train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0]) #x random trees
]

# Assuming X_train, y_train, X_test are already defined in your environment
voting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')

classification_report_with_accuracy_score(y_test, test_predictions)
'''

"\n# This is just used to get the model with the best classifiers\nmodels = [\n    ('logreg', train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0]), #logistic regression\n    ('rf', train_RF_classifier(X_train, y_train, X_test, cat_cols)[0]), #random forest\n    ('xrt', train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0]) #x random trees\n]\n\n# Assuming X_train, y_train, X_test are already defined in your environment\nvoting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')\n\nclassification_report_with_accuracy_score(y_test, test_predictions)\n"

In [18]:
X_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,season,time_of_day
107615,Stray,Normal / Behavior,Cat,Intact Female,1.0,9,2021,non-black,0,Summer,Morning
13002,Stray,Injured,Dog,Intact Male,24.0,3,2014,non-black,1,Summer,Morning
32513,Public Assist,Normal / Behavior,Dog,Intact Female,12.0,4,2019,non-black,1,Summer,Night
79497,Stray,Normal / Behavior,Cat,Intact Female,1.0,9,2015,non-black,1,Summer,Evening
88489,Stray,Normal / Behavior,Dog,Intact Female,4.0,2,2017,black,0,Spring,Morning


In [19]:
# assuming these are pipelines or models you've already trained/tuned
'''
voting_clf = VotingClassifier(
    estimators=[
        ('xgb', xgb_model),      # best XGBoost model
        ('cat', catboost_model), # best CatBoost model
        ('logreg', log_reg_model),
        ('rf', rf_model),
        ('nb', nb_model)
    ],
    voting='soft',
  #  weights=[3, 2.5, 2, 1.5, 1],  # optional: weight more accurate models
    n_jobs=-1
)

voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)

# Evaluate
from sklearn.metrics import balanced_accuracy_score
print("Ensemble Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
'''


'\nvoting_clf = VotingClassifier(\n    estimators=[\n        (\'xgb\', xgb_model),      # best XGBoost model\n        (\'cat\', catboost_model), # best CatBoost model\n        (\'logreg\', log_reg_model),\n        (\'rf\', rf_model),\n        (\'nb\', nb_model)\n    ],\n    voting=\'soft\',\n  #  weights=[3, 2.5, 2, 1.5, 1],  # optional: weight more accurate models\n    n_jobs=-1\n)\n\nvoting_clf.fit(X_train, y_train)\ny_pred = voting_clf.predict(X_test)\n\n# Evaluate\nfrom sklearn.metrics import balanced_accuracy_score\nprint("Ensemble Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))\n'

In [20]:
# used for extra valdiation to find the best thresholds to maximize recall per class
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [21]:
'''

X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# Train on sub-training set
logreg_CV = train_logreg_classifier(X_train_sub, y_train_sub, X_val, cat_cols, num_cols)[0]
rf_CV = train_RF_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]
xrt_CV = train_XRT_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]
xg = train_XG_classifier(X_train_sub, y_train_sub, X_val, [], cat_cols)[0]

# Get predicted probabilities on validation set
log_probs_CV = logreg_CV.predict_proba(X_test)
rf_probs_CV = rf_CV.predict_proba(X_test)
xrt_probs_CV = xrt_CV.predict_proba(X_test)
xg_probs_CV = xg.predict_proba(X_test)

# Average the probabilities
avg_probs_val = (log_probs_CV + rf_probs_CV + xrt_probs_CV + xg_probs_CV) / 4


from itertools import product

# Define classes (adjust as needed)
classes = [0, 1, 2, 3, 4]

# Threshold options for each class
threshold_grid = {
    0: [0.5, 0.6, 0.7, 0.8],
    1: [0.05, 0.1, 0.2],
    2: [0.1, 0.2, 0.3],
    3: [0.2, 0.3, 0.4],
    4: [0.2, 0.3, 0.4]
}

# Generate all threshold combinations
all_thresholds = list(product(*threshold_grid.values()))

# Tune thresholds
best_score = -1
best_thresholds = None

for thresh_combo in all_thresholds:
    thresholds = dict(zip(classes, thresh_combo))
    preds = []

    for probs in avg_probs_val:
        passed = [cls for cls, prob in enumerate(probs) if prob >= thresholds.get(cls, 1.0)]
        if passed:
            chosen = max(passed, key=lambda cls: probs[cls])
        else:
            chosen = np.argmax(probs)
        preds.append(chosen)

    score = balanced_accuracy_score(y_val, preds)

    if score > best_score:
        best_score = score
        best_thresholds = thresholds

print("Best thresholds:", best_thresholds)
print("Best balanced accuracy on validation:", best_score)
'''

'\n\nX_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)\n\n# Train on sub-training set\nlogreg_CV = train_logreg_classifier(X_train_sub, y_train_sub, X_val, cat_cols, num_cols)[0]\nrf_CV = train_RF_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]\nxrt_CV = train_XRT_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]\nxg = train_XG_classifier(X_train_sub, y_train_sub, X_val, [], cat_cols)[0]\n\n# Get predicted probabilities on validation set\nlog_probs_CV = logreg_CV.predict_proba(X_test)\nrf_probs_CV = rf_CV.predict_proba(X_test)\nxrt_probs_CV = xrt_CV.predict_proba(X_test)\nxg_probs_CV = xg.predict_proba(X_test)\n\n# Average the probabilities\navg_probs_val = (log_probs_CV + rf_probs_CV + xrt_probs_CV + xg_probs_CV) / 4\n\n\nfrom itertools import product\n\n# Define classes (adjust as needed)\nclasses = [0, 1, 2, 3, 4]\n\n# Threshold options for each class\nthreshold_grid = {\n    0: [0.5, 0.6, 0.

In [22]:
df_train.head()


,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,outcome_type,season,time_of_day
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,non-black,0,Return to Owner,Summer,Afternoon
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,non-black,1,Return to Owner,Spring,Evening
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,9,2022,non-black,0,Transfer,Spring,Night
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,non-black,1,Return to Owner,Winter,Afternoon
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,black,1,Return to Owner,Spring,Morning


In [23]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# manual voting

#def manual_voting(models):
logreg = train_logreg_classifier(X_train, df_train['outcome_type'], df_test, cat_cols, num_cols)[0] #logistic regression
rf = train_RF_classifier(df_train.drop(columns=['outcome_type']), df_train['outcome_type'], df_test, cat_cols)[0] #random forest
xrt = train_XRT_classifier(df_train.drop(columns=['outcome_type']), df_train['outcome_type'], df_test, cat_cols)[0] #x random trees
xg = train_XG_classifier(df_train.drop(columns=['outcome_type']), df_train['outcome_type'], df_test, [], cat_cols)[0]
df_train = bucket_seasons(df_train)
df_test  = bucket_seasons(df_test)

# Separate target
y_raw_full = df_train['outcome_type']
X_full = df_train.drop(columns=['outcome_type', 'is_mix', 'intake_hour', 'intake_month'])

# === [2] Label encode outcome_type for reporting and consistency ===
le = LabelEncoder()
y_encoded = le.fit_transform(y_raw_full)
print("Encoding mapping:", dict(zip(le.classes_, range(len(le.classes_)))))

# === [3] Identify rare classes (< 5% frequency, for logging/debugging) ===
rare_classes = [
    label for label, count in pd.Series(y_encoded).value_counts().items()
    if count < 0.05 * len(y_encoded)
]
print("Rare classes (by encoded value):")
for cls in rare_classes:
    print(f"  {cls}: {le.classes_[cls]}")

# === [4] DATA SPLITTING ===
X_train, X_temp, y_train_raw, y_temp_raw = train_test_split(
    X_full, y_raw_full, test_size=0.35, stratify=y_raw_full, random_state=42
)

X_val, X_test, y_val_raw, y_test_raw = train_test_split(
    X_temp, y_temp_raw, test_size=0.57, stratify=y_temp_raw, random_state=42
)
# Results in approx: 65% train / 15% val / 20% test

# === [5] CATEGORICAL FEATURE ENCODING ===
categorical_cols = ['intake_type', 'intake_condition', 'animal_type',
                    'sex_upon_intake', 'season', 'breed', 'primary_color']

for col in categorical_cols:
    # Convert training to category
    X_train[col] = X_train[col].astype("category")

    # Apply training categories to val/test
    X_val[col] = pd.Categorical(X_val[col], categories=X_train[col].cat.categories)
    X_test[col] = pd.Categorical(X_test[col], categories=X_train[col].cat.categories)

    # Convert to integer codes (unknowns -> -1)
    X_train[col] = X_train[col].cat.codes
    X_val[col] = X_val[col].cat.codes
    X_test[col] = X_test[col].cat.codes

# Save raw training version for analysis if needed
raw_X_train = X_train.copy()

# Ensure test set has all expected columns
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# === [6] DEFINE DEFAULT THRESHOLDS ===
thresholds = {
    'Adoption': 0.60,
    'Died': 0.10,
    'Euthanasia': 0.10,
    'Return to Owner': 0.15,
    'Transfer': 0.55
}

# === [7] TRAIN OVR MODEL AND PREDICT ===

ovr = train_ovr_xg_classifier(
    X_train=X_train,
    y_train_raw=y_train_raw,
    X_test=X_test,  # Change to X_val if validating
    thresholds=thresholds,
    X_val=X_val,
    y_val_raw=y_val_raw
)[0]


log_probs = logreg.predict_proba(X_test)
rf_probs = rf.predict_proba(X_test)
xrt_probs = xrt.predict_proba(X_test)
xg_probs = xg.predict_proba(X_test)
ovr_probs = xg.predict_proba(X_test)

avg_probs = (log_probs + rf_probs + xrt_probs + xg_probs + ovr_probs ) / 5

'''
avg_probs = (
    0.1667 * log_probs +
    0.1667 * rf_probs +
    0.1667 * xrt_probs +
    0.5    * xg_progs
) 

'''

het_thresholds = {
    0: 0.60,  # Adopted
    1: 0.20,  # Died
    2: 0.30,  # Euth
    3: 0.40,  # RTO
    4: 0.30   # Transfer
}

#le.fit_transform(y_train)

custom_preds = []

for probs in avg_probs:
    # Find which classes exceed their thresholds
    passed = [cls for cls, prob in enumerate(probs) if prob >= het_thresholds.get(cls, 1.0)]
    
    if passed:
        # Among the passed, pick the class with the highest probability
        chosen = max(passed, key=lambda cls: probs[cls])
    else:
        # Default to the class with highest average prob
        chosen = np.argmax(probs)
        
    custom_preds.append(chosen)

test_preds = np.array(custom_preds)
test_preds_labels = le.inverse_transform(custom_preds)

classification_report_with_accuracy_score(y_test, test_preds_labels)
#save_predictions(test_preds_labels, 'voting')

NameError: name 'train_logreg_classifier' is not defined

In [ ]:
'''
# Predict from each already-trained model
log_preds = logreg.predict(X_test)
rf_preds = rf.predict(X_test)
xrt_preds = xrt.predict(X_test)

# Combine predictions manually, e.g. majority vote
from scipy.stats import mode
all_preds = np.vstack([log_preds, rf_preds, xrt_preds])
final_preds, _ = mode(all_preds, axis=0)

'''